In [1]:
import sys
sys.path.append('/scratch/oem214/vanilla-rtrl/')
from cluster import submit_jobs
from continual_learning import *
from core import *
from dynamics import *
from functions import *
from gen_data import *
from learning_algorithms import *
from optimizers import *
from plotting import *
from wrappers import *

IndentationError: unexpected indent (write_main_script.py, line 4)

In [2]:
### --- Set up all configs --- ###
from itertools import product
n_seeds = 10
macro_configs = config_generator(param_1=[1, 2],
                                 param_2=[10, 100])
micro_configs = tuple(product(macro_configs, list(range(n_seeds))))

### --- Select particular config --- ###
try:
    i_job = int(os.environ['SLURM_ARRAY_TASK_ID']) - 1
except KeyError:
    i_job = 0
params, i_seed = micro_configs[i_job]
i_config = i_job//n_seeds

new_random_seed_per_condition = True
if new_random_seed_per_condition:
    np.random.seed(i_job)
else: #Match random seeds across conditions
    np.random.seed(i_seed)

In [5]:
###Truncate file above
file_name = 'pipeline_test'
get_ipython().run_cell_magic('javascript', '', 'IPython.notebook.save_notebook()')
get_ipython().system('jupyter nbconvert --to script --no-prompt {}.ipynb'.format(file_name))
get_ipython().system('awk "/###Truncate/ {{exit}} {{print}}" {}.py'.format(file_name))
get_ipython().system('sed -i "/###Truncate/Q" {}.py'.format(file_name))
get_ipython().system('mv {}.py ../cluster_main_scripts'.format(file_name))

<IPython.core.display.Javascript object>

[NbConvertApp] Converting notebook pipeline_test.ipynb to script
[NbConvertApp] Writing 1500 bytes to pipeline_test.py
awk: 1: unexpected character '.'


In [ ]:
###Submit job to cluster
n_jobs = len(micro_configs)

write_